In [1]:
import numpy as np, pandas as pd, random
from tqdm import tqdm

# --------------------------------------------------------
# 1) carica
# --------------------------------------------------------
df = pd.read_parquet("positions.parquet")

# --------------------------------------------------------
# 3) pesi
# --------------------------------------------------------
# w_path, w_wall, w_goal, w_deg, w_gate = 0.1, 0.2, 0.2, 1.5, 0.1
w_path, w_wall, w_goal, w_deg, w_gate = 0.5, 0.1, 0.15, 0.2, 0.05

# --------------------------------------------------------
# 4) score (minore è migliore)
# --------------------------------------------------------
df["total_score"] = (
      w_path * df.path_distance        #  + penalità: cammino lungo
    + w_goal * df.distance_to_goal        #  + penalità: lontano dal goal
    - w_deg  * df.degree         #  + penalità: vicolo cieco
    - w_wall * df.distance_from_wall        #  – bonus: punti “larghi”, lontani dai muri
    + w_gate * df.distance_to_gate
)

# quick sanity
print(df.nsmallest(3,"total_score")[["x","y","degree","total_score"]])
print(df.nlargest (3,"total_score")[["x","y","degree","total_score"]])

# --------------------------------------------------------
# 5) genera 200k coppie (50 % facili / 50 % difficili)
# --------------------------------------------------------
from tqdm import tqdm
import numpy as np
import pandas as pd

def sample_pairs(
        df: pd.DataFrame,
        num_pairs: int        = 200_000,
        hard_ratio: float     = 0.7,
        thresh: float         = 0.05,
        min_delta_deg: float  = 0.001,     # <- NOVITÀ: differenza minima di degree
        max_trials: int       = 50_000_000   # per evitare loop infiniti
    ):
    """
    Estrae `num_pairs` coppie (x_better,y_better,x_worse,y_worse)
    rispettando:
      • hard_ratio      → % di coppie 'difficili' (score simili)
      • thresh          → soglia fra facile / difficile su total_score
      • min_delta_deg   → scarto minimo assoluto su colonna 'degree'
    Restituisce una lista di tuple di lunghezza 5 (… , 1) compatibili
    con il tuo formato.
    """

    idx       = df.index.to_numpy()
    scores    = df["total_score"].to_numpy()
    degrees   = df["degree"].to_numpy()          # cache per velocità
    pairs     = set()
    n_hard    = int(num_pairs * hard_ratio)

    pbar = tqdm(total=num_pairs, desc="sampling pairs")
    trials = 0
    while len(pairs) < num_pairs and trials < max_trials:
        trials += 1

        i, j = np.random.choice(idx, 2, replace=False)
        si, sj = scores[i], scores[j]

        if si == sj:          # identico score ⇒ salta subito
            continue

        # -----------------------------------------  
        # 1) vincolo sullo score facile / difficile
        # -----------------------------------------
        better, worse = (i, j) if si < sj else (j, i)   # score minore = migliore
        delta_score   = abs(si - sj)
        want_hard     = len(pairs) < n_hard
        if  (want_hard  and delta_score >= thresh) or \
            (not want_hard and delta_score <  thresh):
            continue        # scorretto per la fascia che stiamo riempiendo

        # -----------------------------------------  
        # 2) vincolo sullo scarto di degree
        # -----------------------------------------
        if abs(degrees[better] - degrees[worse]) < min_delta_deg:
            continue        # differenza troppo piccola → poco informativa

        # -----------------------------------------  
        # 3) aggiungi la coppia
        # -----------------------------------------
        pair = (df.at[better,"x"], df.at[better,"y"],
                df.at[worse, "x"], df.at[worse, "y"], 1)

        if pair not in pairs:
            pairs.add(pair)
            pbar.update(1)

    pbar.close()

    if len(pairs) < num_pairs:
        print(f"⚠️  solo {len(pairs)} coppie generate dopo {trials} tentativi "
              f"(probabilmente min_delta_deg è troppo alto).")

    return list(pairs)

def sample_same_row_pairs(df, n_pairs, min_dx=0.25):
    pairs = []
    grouped = df.groupby(pd.cut(df.y, bins=np.arange(0,1.01,0.1)))
    while len(pairs) < n_pairs:
        _, g = random.choice(list(grouped))
        if len(g) < 2: continue
        a, b = g.sample(2).itertuples()
        if abs(a.x - b.x) < min_dx: continue
        better, worse = (a,b) if a.total_score < b.total_score else (b,a)
        pairs.append((better.x,better.y,worse.x,worse.y,1))
    return pairs


n_total   = 200_000
pairs_row = sample_same_row_pairs(df, int(n_total*0.4))
pairs_mix = sample_pairs(df, n_total - len(pairs_row), hard_ratio=0.7)
pairs = pairs_row + pairs_mix


# --------------------------------------------------------
# 6) salva
# --------------------------------------------------------
pd.DataFrame(pairs,
    columns=["x_better","y_better","x_worse","y_worse","preference"]
).to_parquet("preferences.parquet", index=False)

print("✓ preferences.parquet salvato con", len(pairs), "coppie")


             x         y    degree  total_score
1620  0.938950  0.852036  0.972222    -0.266331
1192  0.929181  0.852465  0.972222    -0.257075
497   0.936651  0.841568  0.972222    -0.255781
             x         y  degree  total_score
268   0.003453  0.024670     0.0     0.694189
1334  0.003833  0.047965     0.0     0.691683
2416  0.084460  0.000196     0.0     0.691088


/var/folders/x6/hhp417nn6td5qzxnvywdjbv00000gp/T/ipykernel_38756/3683343305.py:108: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(pd.cut(df.y, bins=np.arange(0,1.01,0.1)))
sampling pairs: 100%|██████████| 120000/120000 [00:50<00:00, 2362.08it/s] 


✓ preferences.parquet salvato con 200000 coppie
